# 2 Прямые методы решения линейных систем 

## 2.4 QR-разложение

In [31]:
import numpy as np

def gramm_schmidt(A):
    p = np.zeros(A.shape)
    q = np.zeros(A.shape)
    r = np.zeros((A.shape[1], A.shape[1]))
    for j in range(A.shape[1]):
        p[:, j] = A[:, j]
        for i in range(j):
            # p[:, j] -= q[:, i] * np.inner(p[:, j], q[:, i])
            # Или, если скалярным произведением пользоваться нельзя,..
            # np.sum() -- точно можно!
            p[:, j] -= q[:, i] * np.sum(p[:, j] * q[:, i])
            # r[i][j] = np.inner(A[:, j], q[:, i])
            r[i][j] = np.sum(A[:, j] * q[:, i])
        # q[:, j] = p[:, j] / np.linalg.norm(p[:, j])
        q[:, j] = p[:, j] / np.sqrt(np.sum(p[:, j] * p[:, j]))
        # r[j][j] = np.inner(A[:, j], q[:, j])
        r[j][j] = np.sum(A[:, j] * q[:, j])
    return q, r

def solve_triangular(M, b):
    '''
    reverse-podstanovka  
    M should be an upper-triangular matrix.   
    '''
    n = b.shape[0]
    x = np.ones(n)
    for i in range(n)[::-1]:
        x[i] = b[i] / M[i][i]
        b -= M.T[i] * x[i]
    return x

def solve(A, b):
    '''
    qr-decomposition + least-squares
    '''
    q, r = gramm_schmidt(A)
    # Это вот тут можно матрицу умножать, да?
    # (q не треугольная, а всего лишь ортогональная)
    # нет тут никакой магии в умножении
    x = solve_triangular(r, q.T @ b)
    return q, r, x

def check_solve_on_random_matrix(m=100, n=100):
    A = np.random.rand(m, n)
    x = np.random.rand(n)
    b = A @ x
    q, r, x_calculated = solve(A, b)

    if (np.allclose(A, q @ r)):
        print('QR-разложение найдено верно')
    else:
        print('QR-разложение найдено НЕВЕРНО')
    print('Норма разницы между полученным решением и ответом = ', np.linalg.norm(x_calculated - x))

    x_np_lstsq = np.linalg.lstsq(A, b)[0]
    print('Норма разницы между полученным решением и ответом lstsq = ', np.linalg.norm(x_np_lstsq - x_calculated))
    
    if n == m:
        x_np_solve = np.linalg.solve(A, b)
        print('Норма разницы между полученным решением и ответом numpy = ', np.linalg.norm(x_np_solve - x_calculated))

In [32]:
check_solve_on_random_matrix()

QR-разложение найдено верно
Норма разницы между полученным решением и ответом =  2.7937897218547425e-11
Норма разницы между полученным решением и ответом lstsq =  2.796505778416829e-11
Норма разницы между полученным решением и ответом numpy =  2.7842159854944645e-11


In [33]:
# И на прямоугольной...
check_solve_on_random_matrix(m=150)

QR-разложение найдено верно
Норма разницы между полученным решением и ответом =  7.287103065500323e-13
Норма разницы между полученным решением и ответом lstsq =  7.170992963792866e-13
